In [1]:
import llm_lib
import os
from transformers import BitsAndBytesConfig
import torch

2024-04-06 12:11:28.933346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


### Loading huggingface llms

In [2]:
load_params = {
    "quantization_config": BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    "torch_dtype": "auto",
    "device_map": "auto"
}

generate_params = {
    "temperature": 0.5,
    "max_new_tokens": 8192,
}
llm = llm_lib.StreamingLLM('hf', './qwen', load_params, generate_params)
# llm = llm_lib.StreamingLLM('hf', './qwen-2-7b/', load_params, generate_params)
# llm = llm_lib.StreamingLLM('hf', 'Qwen/Qwen1.5-7B-Chat-GPTQ-Int4', load_params, generate_params)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
llm.llm.save_pretrained('./qwen')

In [3]:
generator = llm.generate('what is the capital of france')
for part in generator:
    print(part)



Paris


In [6]:
llm.llm('The center of the universe ')

KeyboardInterrupt: 

In [ ]:
llm.chat_history

In [3]:
generator = llm.generate()
for part in generator:
    print(part, end='')

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
what is the capital of france<|im_end|>
<|im_start|>assistant
Paris<|im_end|>

In [5]:
x = 'asdfasdfabc'
y = 'abc'

x[:x.rindex(y)]

'asdfasdf'

### Loading ggml llms

In [5]:
load_params = {
    "n_threads":int(os.cpu_count() * 0.8),
    "n_gpu_layers":20,
    "n_ctx":8192,
}
llm = llm_lib.StreamingLLM('ggml', './qwen-2-7b/ggml-model-Q4_K_M.gguf', load_params, None)

llama_model_loader: loaded meta data with 20 key-value pairs and 387 tensors from ./qwen-2-7b/ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = qwen-2-7b
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 32
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 32
llama_model_loader: - kv   7:              qwen2.attention.head_count_kv u3

IndexError: index out of range

## 2. Langchain file loaders

In [1]:
from langchain_community.document_loaders import UnstructuredFileLoader

In [2]:
loader = UnstructuredFileLoader('../notebooks/data/Sophia+CV+2021.pdf', )

In [3]:
loader.load()

AttributeError: module 'cv2.dnn' has no attribute 'DictValue'

In [ ]:
llm.generate()

In [2]:
llm = llm_lib.construct_ggml_model("qwen-2-7b/ggml-model-Q4_K_M.gguf", context_size=8192, stream=True)

llama_model_loader: loaded meta data with 20 key-value pairs and 387 tensors from qwen-2-7b/ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))


llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = qwen-2-7b
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 32
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 32
llama_model_loader: - kv   7:              qwen2.attention.head_count_kv u32              = 32
llama_model_loader: - kv   8:                       qwen2.rope.freq_base f32              = 1000000.000000
llama_model_loade

In [3]:
llm('hello')


llama_print_timings:        load time =    6906.55 ms
llama_print_timings:      sample time =       0.34 ms /     1 runs   (    0.34 ms per token,  2923.98 tokens per second)
llama_print_timings: prompt eval time =    6906.34 ms /     2 tokens ( 3453.17 ms per token,     0.29 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    6914.30 ms /     3 tokens


{'id': 'cmpl-30355c0f-27cc-4993-8033-10c8d8fa5238',
 'object': 'text_completion',
 'created': 1712266952,
 'model': 'qwen-2-7b/ggml-model-Q4_K_M.gguf',
 'choices': [{'text': '',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 2, 'completion_tokens': 0, 'total_tokens': 2}}

In [5]:
chat_history = llm_lib.ChatHistory.model_validate({'history': [{'role': 'user', 'content': 'hello'}]})

In [10]:
chat_history.model_dump()['history']

[{'role': 'user', 'content': 'hello'}]

In [22]:
chat_history.history.append(llm_lib.ChatMessage.model_validate({'role': 'assistant', 'content': 'hello2'}))
chat_history

ChatHistory(history=[ChatMessage(role='user', content='hello'), {'role': 'user', 'content': 'hello2'}, {'role': 'user', 'content': 'hello2'}, {'role': 'user', 'conten2t': 'hello2'}, {'role': 'user2', 'conten2t': 'hello2'}, {'rolesfs': 'user2', 'conten2t': 'hello2'}, ChatMessage(role='user', content='hello2'), ChatMessage(role='assistant', content='hello2')])